In [1]:
import json
import hydra
from omegaconf import OmegaConf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.pipeline import make_union, make_pipeline

from mllib.transformers import *
from src.pipelines.artifacts import ParsePlayerData
from src.utils.utils import print_config

In [2]:
# Generate index
TRAIN_FILE = "data/train.csv"
PLAYERS_FILE = "data/players.csv"
VAL_START_DATE = 20210415
TARGETS = ["target1", "target2", "target3", "target4"]


In [3]:
raw_data = pd.read_csv(TRAIN_FILE)
tr = raw_data.loc[raw_data.date < VAL_START_DATE]
val = raw_data.loc[raw_data.date >= VAL_START_DATE]
print(raw_data.shape, val.shape)

(1216, 12) (16, 12)


In [4]:
from src.pipelines.artifacts import ParseJsonField

In [5]:
# parser = ParseJsonField('date', 'rosters', use_cols=['status', 'statusCode'])
# roster_df = parser.transform(raw_data)

In [6]:
roster_2021 = pd.read_csv(PLAYERS_FILE)
roster_2021 = roster_2021.loc[roster_2021.playerForTestSetAndFuturePreds == True]
target_enc = ParsePlayerData("nextDayPlayerEngagement", TARGETS)
tr_index = target_enc.fit_transform(tr).reset_index(drop=False)
tr_index = tr_index.loc[tr_index.playerId.isin(roster_2021.playerId.astype(str))]
# tr_index['debutdate'] = tr_index.map()
vl_index = target_enc.fit_transform(val).reset_index(drop=False)
vl_index = vl_index.loc[vl_index.playerId.isin(roster_2021.playerId.astype(str))]
tr_index.to_csv("data/tr_index_small.csv", index=False)
vl_index.to_csv("data/vl_index_small.csv", index=False)


100%|██████████| 16/16 [00:00<00:00, 62.00it/s]


In [7]:
tr_index = pd.read_csv("data/tr_index_small.csv")
vl_index = pd.read_csv("data/vl_index_small.csv")
tr_index.shape, vl_index.shape

((1424400, 6), (18992, 6))

In [8]:
device = 'gpu'
artifacts_path = 'data/artifacts/full'
feature_pipeline = make_union(
    LagN('date', 'playerId', [0, 1, 2, 3], f'{artifacts_path}/train_plscores3.pkl', 'data/', N=1, skip=0, device=device),
    LagN('date', 'playerId', [0, 1, 2, 3], f'{artifacts_path}/train_plscores3.pkl', 'data/', N=2, skip=0, device=device),
    LagN('date', 'playerId', [0, 1, 2, 3], f'{artifacts_path}/train_plscores1.pkl', 'data/', N=1, skip=0, device=device),
    LagN('date', 'playerId', [4], f'{artifacts_path}/train_plscores1.pkl', 'data/', N=1, skip=0, device=device),
    LagN('date', 'playerId', [0, 1, 2, 3], f'{artifacts_path}/train_plscores1.pkl', 'data/', N=2, skip=0, device=device),
    LagN('date', 'playerId', [4], f'{artifacts_path}/train_plscores1.pkl', 'data/', N=2, skip=0, device=device),
    *[
        LagN('date', 'playerId', list(range(i*4, (i+1)*4)), f'{artifacts_path}/train_plscores2.pkl', 'data/', N=j+1, skip=0, device=device)
        for j in range(2) for i in range(6)
     ],
    *[
        ExpandingMean('date', 'playerId', list(range(i*4, (i+1)*4)), f'{artifacts_path}/train_plscores2.pkl', 'data/', N=j, skip=0, device=device)
        for j in [10, 30, 300] for i in range(6)
     ],
    *[
        ExpandingSum('date', 'playerId', list(range(i*4, (i+1)*4)), f'{artifacts_path}/train_plscores2.pkl', 'data/', N=j, skip=0, device=device)
        for j in [10, 30, 300] for i in range(6)
     ],
    *[
        LagN('date', 'playerId', list(range(i*4, (i+1)*4)), f'{artifacts_path}/train_plscores4.pkl', 'data/', N=j+1, skip=0, device=device)
        for j in range(2) for i in range(3)
     ],
    *[
        ExpandingMean('date', 'playerId', list(range(i*4, (i+1)*4)), f'{artifacts_path}/train_plscores4.pkl', 'data/', N=j, skip=0, device=device)
        for j in [10, 30, 300] for i in range(3)
     ],
    *[
        ExpandingSum('date', 'playerId', list(range(i*4, (i+1)*4)), f'{artifacts_path}/train_plscores4.pkl', 'data/', N=j, skip=0, device=device)
        for j in [10, 30, 300] for i in range(3)
     ],
    *[
        LagN('date', 'playerId', [12, 13, 14], f'{artifacts_path}/train_plscores4.pkl', 'data/', N=j+1, skip=0, device=device)
        for j in range(2)
     ],
    *[
        ExpandingMean('date', 'playerId', [12, 13, 14], f'{artifacts_path}/train_plscores4.pkl', 'data/', N=j, skip=0, device=device)
        for j in [10, 30, 300]
     ],
    *[
        ExpandingSum('date', 'playerId', [12, 13, 14], f'{artifacts_path}/train_plscores4.pkl', 'data/', N=j, skip=0, device=device)
        for j in [10, 30, 300]
     ],
    *[
        LagN('date', 'playerId', list(range(i*4, (i+1)*4)), f'{artifacts_path}/train_plscores5.pkl', 'data/', N=1, skip=0, device=device)
        for i in range(5)
     ],
    LagN('date', 'playerId', [20], f'{artifacts_path}/train_plscores5.pkl', 'data/', N=1, skip=0, device=device),
    ExpandingMean('date', 'playerId', list(range(0, 4)), f'{artifacts_path}/train_targets.pkl', 'data/', N=10, skip=3, device=device), 
    ExpandingMean('date', 'playerId', list(range(0, 4)), f'{artifacts_path}/train_targets.pkl', 'data/', N=28, skip=3, device=device), 
    ExpandingMean('date', 'playerId', list(range(0, 4)), f'{artifacts_path}/train_targets.pkl', 'data/', N=365, skip=3, device=device), 
    ExpandingMean('date', 'playerId', list(range(0, 4)), f'{artifacts_path}/train_targets.pkl', 'data/', N=1500, skip=3, device=device), 
    ExpandingMax('date', 'playerId', list(range(0, 4)), f'{artifacts_path}/train_targets.pkl', 'data/', N=28, skip=3, device=device), 
    ExpandingQ75('date', 'playerId', list(range(0, 4)), f'{artifacts_path}/train_targets.pkl', 'data/', N=28, skip=3, device=device), 
    ExpandingQ25('date', 'playerId', list(range(0, 4)), f'{artifacts_path}/train_targets.pkl', 'data/', N=28, skip=3, device=device), 
    ExpandingQ05('date', 'playerId', list(range(0, 4)), f'{artifacts_path}/train_targets.pkl', 'data/', N=28, skip=3, device=device),
    LagN('date', 'playerId', [0], f'{artifacts_path}/train_awards.pkl', 'data/', fill_value=np.nan, N=1, skip=0, device=device),
    ExpandingCount('date', 'playerId', [0], f'{artifacts_path}/train_awards.pkl', 'data/', fill_value=np.nan, N=365, skip=0, device=device),
    LagN('date', 'playerId', [0], f'{artifacts_path}/train_transactions.pkl', 'data/', fill_value=np.nan, N=1, skip=0, device=device),
    LagN('date', 'playerId', [0], f'{artifacts_path}/train_rosters.pkl', 'data/', fill_value=np.nan, N=1, skip=0, device=device),
    verbose=10
)

In [9]:
%%time
X_tr1 = feature_pipeline.transform(tr_index)

CPU times: user 4min 49s, sys: 15 s, total: 5min 4s
Wall time: 5min 4s


In [10]:
%%time
X_vl1 = feature_pipeline.transform(vl_index)

CPU times: user 15.4 s, sys: 1.31 s, total: 16.7 s
Wall time: 16.7 s


In [11]:
X_tr1.shape

(1424400, 387)

In [29]:
import lightgbm as lgb
targets = ['target1', 'target2', 'target3', 'target4']
y_tr = tr_index[targets].values
y_vl = vl_index[targets].values
# print(np.unique(X_tra[:, 235]))
tr1 = lgb.Dataset(X_tr1, y_tr[:, 0], categorical_feature=[13, 14, 15, 16, ])
tr2 = lgb.Dataset(X_tr1, y_tr[:, 1], categorical_feature=[13, 14, 15, 16, ])
tr3 = lgb.Dataset(X_tr1, y_tr[:, 2], categorical_feature=[13, 14, 15, 16, ])
tr4 = lgb.Dataset(X_tr1, y_tr[:, 3], categorical_feature=[13, 14, 15, 16, ])

vl1 = lgb.Dataset(X_vl1, y_vl[:, 0], reference=tr1)
vl2 = lgb.Dataset(X_vl1, y_vl[:, 1], reference=tr2)
vl3 = lgb.Dataset(X_vl1, y_vl[:, 2], reference=tr3)
vl4 = lgb.Dataset(X_vl1, y_vl[:, 3], reference=tr4)

# params = {
#     'n_estimators': 4000,
#     'learning_rate': 0.08,
#     'num_leaves': 31,
#     'colsample_bytree': 0.3,
#     'subsample': 0.5,
#     'reg_alpha': 0.1,
#     'reg_lambda': 0.1,
#     'max_bin': 255,
#     'objective': 'mae',
#     'metric': 'mae'
# }

params = {
    'n_estimators': 4000,
    'learning_rate': 0.05,
    'num_leaves': 255,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'colsample_bytree': 0.5,
    'subsample': 0.95,
    'bagging_freq': 1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'extra_trees': False,
    'max_bin': 127,
    #'device': 'gpu',
    #'gpu_use_dp': False,
    #'gpu_device_id': 0,
    'boost_from_average': True,
    'reg_sqrt': True,
    'objective': 'mae',
    'metric': 'mae',
    'verbose': -1,
    'seed': 123478659,
    'min_data_per_group': 10,
    'cat_l2': 1,
    'cat_smooth': 10,
    'num_threads': 16
}
bst1 = lgb.train(params, tr1, valid_sets=[vl1], early_stopping_rounds=200, verbose_eval=50)


/home/mohsin_okcredit_in/anaconda3/envs/setienv/lib/python3.8/site-packages/lightgbm/engine.py:154: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/mohsin_okcredit_in/anaconda3/envs/setienv/lib/python3.8/site-packages/lightgbm/basic.py:1991: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mohsin_okcredit_in/anaconda3/envs/setienv/lib/python3.8/site-packages/lightgbm/basic.py:1994: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [13, 14, 15, 16]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mohsin_okcredit_in/anaconda3/envs/setienv/lib/python3.8/site-packages/lightgbm/basic.py:1722: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/mohsin_okcredit_in/anaconda3/envs

Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 1.1701
[100]	valid_0's l1: 1.12442
[150]	valid_0's l1: 1.11326
[200]	valid_0's l1: 1.11069
[250]	valid_0's l1: 1.10889
[300]	valid_0's l1: 1.10821
[350]	valid_0's l1: 1.10803
[400]	valid_0's l1: 1.10683
[450]	valid_0's l1: 1.10631
[500]	valid_0's l1: 1.10591
[550]	valid_0's l1: 1.10558
[600]	valid_0's l1: 1.10537
[650]	valid_0's l1: 1.10537
[700]	valid_0's l1: 1.10527
[750]	valid_0's l1: 1.105
[800]	valid_0's l1: 1.10479
[850]	valid_0's l1: 1.10477
[900]	valid_0's l1: 1.10468
[950]	valid_0's l1: 1.10463
[1000]	valid_0's l1: 1.1044
[1050]	valid_0's l1: 1.10431
[1100]	valid_0's l1: 1.10416
[1150]	valid_0's l1: 1.10373
[1200]	valid_0's l1: 1.10343
[1250]	valid_0's l1: 1.10323
[1300]	valid_0's l1: 1.10325
[1350]	valid_0's l1: 1.10325
[1400]	valid_0's l1: 1.10292
[1450]	valid_0's l1: 1.10305
[1500]	valid_0's l1: 1.10311
[1550]	valid_0's l1: 1.1027
[1600]	valid_0's l1: 1.10271
[1650]	valid_0's l1: 1.10247
[1700]

In [30]:
bst2 = lgb.train(params, tr2, valid_sets=[vl2], early_stopping_rounds=200, verbose_eval=50)

Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 2.08182
[100]	valid_0's l1: 2.03233
[150]	valid_0's l1: 2.01875
[200]	valid_0's l1: 2.01287
[250]	valid_0's l1: 2.01065
[300]	valid_0's l1: 2.00737
[350]	valid_0's l1: 2.00318
[400]	valid_0's l1: 2.00111
[450]	valid_0's l1: 2.00007
[500]	valid_0's l1: 1.99933
[550]	valid_0's l1: 1.99796
[600]	valid_0's l1: 1.99697
[650]	valid_0's l1: 1.99646
[700]	valid_0's l1: 1.99629
[750]	valid_0's l1: 1.99595
[800]	valid_0's l1: 1.99587
[850]	valid_0's l1: 1.99521
[900]	valid_0's l1: 1.99515
[950]	valid_0's l1: 1.99457
[1000]	valid_0's l1: 1.99498
[1050]	valid_0's l1: 1.99454
[1100]	valid_0's l1: 1.99417
[1150]	valid_0's l1: 1.9939
[1200]	valid_0's l1: 1.99362
[1250]	valid_0's l1: 1.9936
[1300]	valid_0's l1: 1.99362
[1350]	valid_0's l1: 1.99314
[1400]	valid_0's l1: 1.99323
[1450]	valid_0's l1: 1.99247
[1500]	valid_0's l1: 1.99203
[1550]	valid_0's l1: 1.99189
[1600]	valid_0's l1: 1.9917
[1650]	valid_0's l1: 1.99085
[170

In [31]:
bst3 = lgb.train(params, tr3, valid_sets=[vl3], early_stopping_rounds=200, verbose_eval=50)

Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 0.911314
[100]	valid_0's l1: 0.896524
[150]	valid_0's l1: 0.893133
[200]	valid_0's l1: 0.892945
[250]	valid_0's l1: 0.892409
[300]	valid_0's l1: 0.89181
[350]	valid_0's l1: 0.891588
[400]	valid_0's l1: 0.89135
[450]	valid_0's l1: 0.891338
[500]	valid_0's l1: 0.891042
[550]	valid_0's l1: 0.890974
[600]	valid_0's l1: 0.890859
[650]	valid_0's l1: 0.890755
[700]	valid_0's l1: 0.890829
[750]	valid_0's l1: 0.890869
[800]	valid_0's l1: 0.890661
[850]	valid_0's l1: 0.890265
[900]	valid_0's l1: 0.889598
[950]	valid_0's l1: 0.88921
[1000]	valid_0's l1: 0.889158
[1050]	valid_0's l1: 0.889152
[1100]	valid_0's l1: 0.888935
[1150]	valid_0's l1: 0.888228
[1200]	valid_0's l1: 0.887406
[1250]	valid_0's l1: 0.887306
[1300]	valid_0's l1: 0.887485
[1350]	valid_0's l1: 0.887093
[1400]	valid_0's l1: 0.886847
[1450]	valid_0's l1: 0.887088
[1500]	valid_0's l1: 0.887414
[1550]	valid_0's l1: 0.887337
Early stopping, best iteration 

In [32]:
bst4 = lgb.train(params, tr4, valid_sets=[vl4], early_stopping_rounds=200, verbose_eval=50)

Training until validation scores don't improve for 200 rounds
[50]	valid_0's l1: 1.48375
[100]	valid_0's l1: 1.43275
[150]	valid_0's l1: 1.4264
[200]	valid_0's l1: 1.42518
[250]	valid_0's l1: 1.42459
[300]	valid_0's l1: 1.42372
[350]	valid_0's l1: 1.42301
[400]	valid_0's l1: 1.42293
[450]	valid_0's l1: 1.42328
[500]	valid_0's l1: 1.42316
Early stopping, best iteration is:
[333]	valid_0's l1: 1.42285


In [33]:
from sklearn.metrics import mean_absolute_error as mae
pred1 = bst1.predict(X_vl1)
pred2 = bst2.predict(X_vl1)
pred3 = bst3.predict(X_vl1)
pred4 = bst4.predict(X_vl1)
preds = np.vstack((pred1, pred2, pred3, pred4)).T
print(mae(y_vl, preds))

1.3468528152749888


In [ ]:
# 1.7930
# 1.7708 - added lag of flags
# 1.7368 - added categorical features from box score
# 1.6982 - added batter scores
# 1.6890       - added pitcher scores
# 1.6892   - added remaining features ( :-( )
# 1.6883     - added pitcher lags 
# 1.6810   - adde batter lags
# 1.6777   - more pitcher lags
# 1.5774 - added target mean (skip - 30 and last 365)
# change hyperparams - colsamplebytree 0.7 --> 0.4; 1.6814 (reverting)
# 1.534 - changed val to from 10 april - 
# 1.517 - added lag3
# 1.5146 - changed hyperparams - colsample to 0.5
# 1.509 - num_leaves 255
# 1.5078  - min_leaf_samples 20
# 1.4884 - added statusCode
# 1.4891 - made it cat
# 1.4795 - removed cat encoding
# 1.4455 - fixed last n expanding mean and added last 10
# 1.4396 - added last 10 stats
# 1.4379 - last 5 innings sum of pitching scores
# 1.4322 - last 5 mean scores
# 1.4278 - expading mean batter scores
# 1.4257 - expanding mean pitcher scores
# 1.4070 - expanding sum - last 10 scores 
# 1.3687 -- changed validation to last 15 days
# 1.3602 - changed target means from 10 days to 15 days 
# 1.3556 - changed from 15 to 30 days
# 1.355 - changes days a bit
# 1.3499 - changed lr to 0.05
# 1.346 - changed hyperparams

In [45]:
bst1.save_model("artifacts/bst1_train_v2.pkl")
bst2.save_model("artifacts/bst2_train_v2.pkl")
bst3.save_model("artifacts/bst3_train_v2.pkl")
bst4.save_model("artifacts/bst4_train_v2.pkl")

In [38]:
pd.read_csv("data/seasons.csv")

,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate
0,2017,2017-04-02,2017-11-01,2017-02-22,2017-04-01,2017-04-02,2017-10-01,2017-07-09,2017-07-11,2017-07-14,2017-10-03,2017-11-01
1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28
2,2019,2019-03-20,2019-10-30,2019-02-21,2019-03-26,2019-03-20,2019-09-29,2019-07-07,2019-07-09,2019-07-11,2019-10-01,2019-10-30
3,2020,2020-07-23,2020-10-28,2020-02-21,2020-07-22,2020-07-23,2020-09-27,2020-08-25,NaN,2020-08-26,2020-09-29,2020-10-28
4,2021,2021-02-28,2021-10-31,2021-02-28,2021-03-30,2021-04-01,2021-10-03,2021-07-11,2021-07-13,2021-07-15,2021-10-04,2021-10-31


In [32]:
# TODO
# Running stats on rank per day